In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization

In [2]:
train = pd.read_csv('train.csv')
label = pd.read_csv('train_label.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')

# 特征工程

In [3]:
test['label'] = -1
train = train.merge(label,on='ID',how='left')
data = pd.concat([train, test])

In [4]:
def get_time_fe(df):
    df['day'] = df.date.apply(lambda x:int(x[8:10]))

    df['hour'] = df.date.apply(lambda x:int(x[11:13]))

    return df

In [5]:
#对时间分箱
def getSeg(x):
    if x >=0 and x<= 6:
        return 1
    elif x>=7 and x<=12:
        return 2
    elif x>=13 and x<=18:
        return 3
    elif x>=19 and x<=23:
        return 4

In [6]:
# hourDF = train_df.groupby(['hour', 'label'])['hour'].count().unstack('label').fillna(0)
# hourDF[[0,1]].plot(kind='bar', stacked=True);

In [7]:
#count统计特征
cross_feature = []
def get_cross_fe(df):
    first_feature = [ 'B2', 'B3']
    second_feature = ['C1','C2','C3','D1']
    for feat_1 in first_feature:
        for feat_2 in second_feature:
            col_name = "cross_" + feat_1 + "_and_" + feat_2
            cross_feature.append(col_name)
            df[col_name] = df[feat_1].astype(str).values + '_' + df[feat_2].astype(str).values
    return df

In [8]:
#获取nunique特征
def get_nunique_1_fe(df):
    adid_nuq = [ 'hour','E1','E14']
    for feat in adid_nuq:
        gp1 = df.groupby('A2')[feat].nunique().reset_index().rename(columns={feat: "A2_%s_nuq_num" % feat})
        gp2 = df.groupby(feat)['A2'].nunique().reset_index().rename(columns={'A2': "%s_A2_nuq_num" % feat})
        df = pd.merge(df, gp1, how='left', on=['A2'])
        df = pd.merge(df, gp2, how='left', on=[feat])
    return df
def get_nunique_2_fe(df):
    adid_nuq = [ 'E1','E14']
    for feat in adid_nuq:
        gp1 = df.groupby('hour')[feat].nunique().reset_index().rename(columns={feat: "hour_%s_nuq_num" % feat})
        gp2 = df.groupby(feat)['hour'].nunique().reset_index().rename(columns={'hour': "%s_hour_nuq_num" % feat})
        df = pd.merge(df, gp1, how='left', on=['hour'])
        df = pd.merge(df, gp2, how='left', on=[feat])
    return df
# def get_nunique_3_fe(df):
#     adid_nuq = [ 'E14']
#     for feat in adid_nuq:
#         gp1 = df.groupby('E1')[feat].nunique().reset_index().rename(columns={feat: "E1_%s_nuq_num" % feat})
#         gp2 = df.groupby(feat)['E1'].nunique().reset_index().rename(columns={'E1': "%s_E1_nuq_num" % feat})
#         df = pd.merge(df, gp1, how='left', on=['E1'])
#         df = pd.merge(df, gp2, how='left', on=[feat])
#     return df

In [9]:
data = get_time_fe(data)
# data['hour_seg'] = data['hour'].apply(lambda x: getSeg(x))
data = get_cross_fe(data)
data = get_nunique_1_fe(data)
data = get_nunique_2_fe(data)
# data = get_nunique_3_fe(data)

In [10]:
#labelencoder()
cate_feature = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','E2','E3','E5','E7','E9','E10','E13','E16','E17','E19','E21','E22']
cross_feature = cross_feature[:15]
cate_features = cate_feature+cross_feature
for item in cate_features:
    data[item] = LabelEncoder().fit_transform(data[item])

In [11]:
def feature_count(data, features=[]):
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i
    try:
        del data[new_feature]
    except:
        pass
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

for i in cross_feature:
    n = data[i].nunique()
    if n > 5:
        data = feature_count(data, [i])
    else:
        print(i, ':', n)

In [12]:
#ratio：类别偏好的ratio比例特征
label_feature =[ 'A2', 'A3']
data_temp = data[label_feature]
df_feature = pd.DataFrame()
data_temp['cnt'] = 1
print('Begin ratio clcik...')
col_type = label_feature.copy()
n = len(col_type)
for i in range(n):
    col_name = "ratio_click_of_" + col_type[i]
    df_feature[col_name] = (
                    data_temp[col_type[i]].map(data_temp[col_type[i]].value_counts()) / len(data) * 100).astype(int)            
data = pd.concat([data, df_feature], axis=1)
print('The end')

Begin ratio clcik...
The end


In [13]:
len(data.D2.unique())

3

In [14]:
train_df = data[data['label'] != -1]
test_df = data[data['label'] == -1]

In [15]:
# sns.stripplot(train_df["label"],train_df["A3"],jitter=True,order=None)
# plt.title("click Vs creative_height");

# 删除不需要的字段

In [16]:
## get train feature
del_feature = ['ID','day','date','label','D2']+cross_feature
features = [i for i in train_df.columns if i not in del_feature]

In [17]:
train_x = train_df[features]
train_y = train_df['label'].values
test = test_df[features]

In [18]:
# params={'booster':'gbtree',
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'max_depth':4,
#     'lambda':10,
        
#     'subsample':0.75,
#     'colsample_bytree':0.75,
#     'min_child_weight':2,
#     'eta': 0.025,
#     'seed':0,

#      'silent':1}

In [25]:
data = xgb.DMatrix(train_x, label = train_y)
AUCbest = -1.
ITERbest = 0
# Comment out any parameter you don't want to test
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):

#     global AUCbest
#     global ITERbest
    

#
# Define all XGboost parameters
#

    paramt = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 8,
              'silent' : True,
              'eval_metric': 'auc',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'seed' : 1001
              }

    folds = 10
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt) )

    xgbc = xgb.cv(
                    paramt,
                    data,
                    num_boost_round = 2500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 100,
                    metrics = 'auc',
                    show_stdv = True
               )


    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
(val_score*2-1)) )
#     if ( val_score > AUCbest ):
#         AUCbest = val_score
#         ITERbest = len(xgbc)

    return (val_score*2) - 1

XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 10.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

XGB_BO.maximize(init_points=21, n_iter=90, acq='ei', xi=0.0)
print (XGB_BO.max)

|   iter    |  target   | colsam... |   gamma   | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 7, 'gamma': 3.3465814379114796, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9530907111416981, 'colsample_bytree': 0.6629742528213466, 'min_child_weight': 16.045712736686664, 'max_delta_step': 2, 'seed': 1001}
 Stopped after 152 iterations with train-auc = 0.799400 val-auc = 0.725741 ( diff = 0.073659 ) train-gini = 0.598799 val-gini = 0.451482
|  1        |  0.4515   |  0.663    |  3.347    |  2.632    |  7.819    |  16.05    |  0.9531   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 9, 'gamma': 8.223833552791579, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample

 Stopped after 79 iterations with train-auc = 0.810988 val-auc = 0.725761 ( diff = 0.085227 ) train-gini = 0.621976 val-gini = 0.451523
|  13       |  0.4515   |  0.42     |  4.458    |  4.734    |  11.25    |  8.578    |  0.6757   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 8, 'gamma': 9.876748024549311, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.4449065839752646, 'colsample_bytree': 0.7510661243203456, 'min_child_weight': 18.134863638347326, 'max_delta_step': 1, 'seed': 1001}
 Stopped after 345 iterations with train-auc = 0.770864 val-auc = 0.722085 ( diff = 0.048779 ) train-gini = 0.541728 val-gini = 0.444170
|  14       |  0.4442   |  0.7511   |  9.877    |  1.466    |  8.408    |  18.13    |  0.4449   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 7, 'gamma': 6.173766632953327, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': T

 Stopped after 1480 iterations with train-auc = 0.766564 val-auc = 0.724824 ( diff = 0.041740 ) train-gini = 0.533128 val-gini = 0.449649
|  26       |  0.4496   |  0.8416   |  0.0523   |  9.646    |  2.004    |  18.92    |  0.9938   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 7, 'gamma': 4.857186832325416, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9851707317773069, 'colsample_bytree': 0.5155143981752177, 'min_child_weight': 7.178024223398172, 'max_delta_step': 0, 'seed': 1001}
 Stopped after 165 iterations with train-auc = 0.800673 val-auc = 0.726543 ( diff = 0.074130 ) train-gini = 0.601346 val-gini = 0.453087
|  27       |  0.4531   |  0.5155   |  4.857    |  0.125    |  7.761    |  7.178    |  0.9852   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 5.62267807710346, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': 

 Stopped after 1068 iterations with train-auc = 0.762784 val-auc = 0.725266 ( diff = 0.037518 ) train-gini = 0.525567 val-gini = 0.450532
|  39       |  0.4505   |  0.4627   |  9.911    |  6.492    |  7.942    |  17.95    |  0.9996   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 2, 'gamma': 0.17300588672760567, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.4524798300497365, 'colsample_bytree': 0.9372935850265974, 'min_child_weight': 4.992825857141887, 'max_delta_step': 3, 'seed': 1001}
 Stopped after 1772 iterations with train-auc = 0.777918 val-auc = 0.723245 ( diff = 0.054673 ) train-gini = 0.555835 val-gini = 0.446490
|  40       |  0.4465   |  0.9373   |  0.173    |  3.538    |  2.227    |  4.993    |  0.4525   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 4, 'gamma': 7.220341345142258, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent

 Stopped after 358 iterations with train-auc = 0.787480 val-auc = 0.726615 ( diff = 0.060865 ) train-gini = 0.574961 val-gini = 0.453230
|  52       |  0.4532   |  0.4828   |  6.416    |  0.8992   |  7.866    |  16.03    |  0.9973   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 9, 'gamma': 7.119511536162171, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9628025932830461, 'colsample_bytree': 0.41223074413642113, 'min_child_weight': 19.95712195491311, 'max_delta_step': 9, 'seed': 1001}
 Stopped after 221 iterations with train-auc = 0.785066 val-auc = 0.726838 ( diff = 0.058228 ) train-gini = 0.570132 val-gini = 0.453677
|  53       |  0.4537   |  0.4122   |  7.12     |  9.973    |  9.218    |  19.96    |  0.9628   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 8.539301414935393, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent':

 Stopped after 1325 iterations with train-auc = 0.785382 val-auc = 0.727559 ( diff = 0.057823 ) train-gini = 0.570764 val-gini = 0.455118
|  65       |  0.4551   |  0.4242   |  7.259    |  7.184    |  6.394    |  15.61    |  0.9943   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 9, 'gamma': 8.291910383126705, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9861629845939814, 'colsample_bytree': 0.4074980271802656, 'min_child_weight': 10.883757867981016, 'max_delta_step': 5, 'seed': 1001}
 Stopped after 421 iterations with train-auc = 0.792278 val-auc = 0.727548 ( diff = 0.064730 ) train-gini = 0.584556 val-gini = 0.455096
|  66       |  0.4551   |  0.4075   |  8.292    |  5.193    |  9.119    |  10.88    |  0.9862   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 7, 'gamma': 7.962801711967972, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent':

 Stopped after 162 iterations with train-auc = 0.801570 val-auc = 0.727968 ( diff = 0.073602 ) train-gini = 0.603139 val-gini = 0.455936
|  78       |  0.4559   |  0.4127   |  7.211    |  9.786    |  11.85    |  9.259    |  0.9858   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 6.161637856952435, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9973818112554299, 'colsample_bytree': 0.4266788263802726, 'min_child_weight': 8.083119500591309, 'max_delta_step': 7, 'seed': 1001}
 Stopped after 132 iterations with train-auc = 0.812134 val-auc = 0.727465 ( diff = 0.084669 ) train-gini = 0.624267 val-gini = 0.454930
|  79       |  0.4549   |  0.4267   |  6.162    |  7.921    |  11.87    |  8.083    |  0.9974   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 8.125212987841332, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent':

 Stopped after 382 iterations with train-auc = 0.797537 val-auc = 0.727369 ( diff = 0.070168 ) train-gini = 0.595074 val-gini = 0.454738
|  91       |  0.4547   |  0.4193   |  9.44     |  7.974    |  11.84    |  7.764    |  0.965    |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 6.680801248350461, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9961225432914325, 'colsample_bytree': 0.400044337541082, 'min_child_weight': 8.72135308233592, 'max_delta_step': 6, 'seed': 1001}
 Stopped after 169 iterations with train-auc = 0.809288 val-auc = 0.727793 ( diff = 0.081494 ) train-gini = 0.618575 val-gini = 0.455586
|  92       |  0.4556   |  0.4      |  6.681    |  6.436    |  11.75    |  8.721    |  0.9961   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 7.5491155036842486, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': 

 Stopped after 269 iterations with train-auc = 0.814855 val-auc = 0.727032 ( diff = 0.087822 ) train-gini = 0.629709 val-gini = 0.454065
|  104      |  0.4541   |  0.431    |  8.257    |  7.493    |  12.0     |  4.27     |  0.9782   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 9.72902684265447, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent': True, 'eval_metric': 'auc', 'subsample': 0.9863772195435319, 'colsample_bytree': 0.42495084299076086, 'min_child_weight': 19.013325169399863, 'max_delta_step': 0, 'seed': 1001}
 Stopped after 498 iterations with train-auc = 0.780829 val-auc = 0.726810 ( diff = 0.054019 ) train-gini = 0.561658 val-gini = 0.453620
|  105      |  0.4536   |  0.425    |  9.729    |  0.1679   |  11.77    |  19.01    |  0.9864   |

 Search parameters (10-fold validation):
 {'booster': 'gbtree', 'max_depth': 11, 'gamma': 8.614058948687404, 'eta': 0.1, 'objective': 'binary:logistic', 'nthread': 8, 'silent'

# 训练参数的选取

In [20]:
# params= {
#    'num_leaves':97,
#     'objective':'binary',
#     'learning_rate':0.02,
#    'max_bin':498,
#    'min_data_in_leaf':10,
#     'feature_fraction':0.164,
#     'subsample':0.8,
#     'subsample_freq':1,
#    'bagging_freq':1,
#    'bagging_seed':0,
#     'min_split_gain':0.000135,
#     'min_child_weight':5,
#     'min_child_samples':30,
#     'bagging_fraction':0.9,
#     'metric': {'binary_logloss', 'auc'},
#    'reg_alpha':3,
#    'reg_lambda':5
# }